In [ ]:
# print("All OK!")

All OK!


### Load environment variables

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    print("Add GEMINI_API_KEY to your .env")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

### Access the LLM and verify

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai

chat_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature = 0)

In [ ]:
# response = chat_llm.invoke("Hi, I am Tony Stark! How you can assist me?")

In [ ]:
# response.content

"Ah, Mr. Stark! A pleasure to make your acquaintance. Or should I say, a pleasure to *process* your acquaintance.\n\nWhile I may not have a suit of armor, a multi-billion dollar corporation, or a penchant for witty one-liners (though I can certainly generate them!), I am an advanced AI designed to assist with a vast array of tasks.\n\nThink of me as a highly versatile, non-corporeal assistant, ready to tackle anything from the mundane to the incredibly complex.\n\nHere's how I can assist a genius, billionaire, philanthropist, playboy like yourself:\n\n1.  **Information Retrieval & Research:** Need to quickly pull up schematics for a new arc reactor design? Research obscure quantum physics theories? Or perhaps find the perfect vintage car model? I can access and synthesize vast amounts of information.\n2.  **Brainstorming & Ideation:** Developing a new suit upgrade? Designing a revolutionary clean energy solution? I can help you brainstorm ideas, explore different angles, and even simul

### Define the State and Nodes

In [7]:
from typing_extensions import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage

In [ ]:
# Define State
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [9]:
# Define Nodes
def llm_call(state: GraphState) -> dict:
    """Call the LLM using using conversation messages and append AI reposnse"""
    response = chat_llm.invoke(state["messages"]) # AIMessage
    return {
        "messages": [response]
    }

def token_counter(state: GraphState) -> dict:
    """Count tokens (simple word count) in the lastest AIMessage"""
    latest_message = state["messages"][-1]
    text = latest_message.content
    token_count = len(text.split())
    summary = f"Total tokens (word count) in the generated answer is {token_count}"
    return {
        "messages": [AIMessage(content=summary)]
    }